# SNAC API grab - related orgs

Decided to separate this from related people since I've finished the people import and am guessing that the orgs will require something very different.

In [47]:
import time
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [32]:
query = """
{
    "command": "read",
    "constellationid": 83834110
}
"""
url = 'https://api.snaccooperative.org/'

In [33]:
# query SNAC for Bond's constellation

response = requests.put(url, query)
print(response)

<Response [200]>


In [34]:
text = response.text
text_json = json.loads(text)

relations = text_json['constellation']['relations']

relations_data = pd.DataFrame([(i['targetConstellation'], i['content'], i['targetEntityType']['term']) for i in relations])
relations_data.columns = ['constellation_id', 'name', 'entity_type']
relations_data.head()

,constellation_id,name,entity_type
0,5532829,American Civil Liberties Union.,corporateBody
1,76111561,American Folklife Center,corporateBody
2,47287903,American Foundation For Equal Rights.,corporateBody
3,33162909,Antioch review.,corporateBody
4,47966548,Atlanta University Center (Ga.),corporateBody


In [35]:
relations_data.entity_type.value_counts()

entity_type
person           155
corporateBody     53
family             3
Name: count, dtype: int64

In [36]:
relations_data[relations_data.entity_type == 'family']

,constellation_id,name,entity_type
10,29280972,Bond family.,family
158,5965928,Rubinstein family.,family
205,70036600,"Wilson, Bill (William Bryan), 1914-1993",family


In [37]:
# think I don't necessarily want to deal with that right now. Bit weird though. Shoutout Bill Wilson.
orgs = relations_data[relations_data.entity_type == 'corporateBody']

In [38]:
orgs.name

0                        American Civil Liberties Union.
1                               American Folklife Center
2                  American Foundation For Equal Rights.
3                                        Antioch review.
4                        Atlanta University Center (Ga.)
5                                  Atlanta Urban League.
7                                     Barnes Foundation.
20                  Brotherhood of Sleeping Car Porters.
25                          Center for Community Change.
26     Central Michigan University. Office of the Pre...
31     Citizens Board of Inquiry into Health Services...
37                 Committee on Appeal for Human Rights.
38      Communist Party of the United States of America.
43                             David Pearson Associates,
47     Democratic National Convention (1968 : Chicago...
48     Democratic National Convention (1984 : San Fra...
49                                Democratic Party (Ga.)
50                             

In [39]:
# lotta weird stuff happening here (why three SEFs?)
# think I want to just scrape everything and then decide what I want to do with them.

In [40]:
orgs.loc[0,:]

constellation_id                            5532829
name                American Civil Liberties Union.
entity_type                           corporateBody
Name: 0, dtype: object

In [41]:
# test query: let's look at the ACLU's response

test_data = {
    "command": "read",
    "constellationid": "5532829"
}

test_query = json.dumps(test_data)
test_response = requests.put(url, test_query)
test_json = json.loads(test_response.text)

In [42]:
test_json['constellation']['nameEntries'][0]['original']

'American Civil Liberties Union'

In [48]:
test_hist = test_json['constellation']['biogHists'][0]['text']

In [50]:
test_text = test_hist.split('</p>')[0]
BeautifulSoup(test_text).get_text()

"Founded in 1920 in New York City by Roger Baldwin and others; the ACLU was an outgrowth of the American Union Against Militarism's National Civil Liberties Bureau, which in 1920 changed its name to the American Civil Liberties Union."

In [51]:
# well I understand why it took so long this thing is freaking huge
# full history is too big to display on our site, we should stick to redirecting

org_data = []

for idx, row in orgs.iterrows():
    
    data = {
        "command": "read",
        "constellationid": row['constellation_id']
    }

    query = json.dumps(data)
    response = requests.put(url, query)
    json_text = json.loads(response.text)

    constellation = json_text['constellation']

    try:
        name = constellation['nameEntries'][0]['original']
    except:
        name = ''

    try:
        hist_snippet = constellation['biogHists'][0]['text']
        hist_snippet = BeautifulSoup(hist_snippet.split('</p>')[0]).get_text()
        hist = f'From Social Networks and Archival Context:{hist_snippet}...'
    except:
        hist = ''

    arkid = constellation['ark']

    info = {'name': name, 'arkid': arkid, 'constellation_id': row['constellation_id'], 'history': hist}

    org_data.append(info)

In [52]:
pd.DataFrame(org_data)

,name,arkid,constellation_id,history
0,American Civil Liberties Union,http://n2t.net/ark:/99166/w65x61pb,5532829,From Social Networks and Archival Context:Foun...
1,Library of Congress. American Folklife Center,http://n2t.net/ark:/99166/w6fg4gv6,76111561,From Social Networks and Archival Context:The ...
2,American Foundation for Equal Rights,http://n2t.net/ark:/99166/w6tj2mm2,47287903,
3,Antioch review.,http://n2t.net/ark:/99166/w6mr0m9g,33162909,From Social Networks and Archival Context:The ...
4,Atlanta University Center (Ga.),http://n2t.net/ark:/99166/w62p06cr,47966548,From Social Networks and Archival Context:The ...
5,Atlanta Urban League,http://n2t.net/ark:/99166/w620066m,1718260,From Social Networks and Archival Context:The ...
6,Barnes foundation,http://n2t.net/ark:/99166/w6xw88fb,37407383,
7,Brotherhood of sleeping car porters,http://n2t.net/ark:/99166/w6nh5hcx,29098008,From Social Networks and Archival Context:\nTh...
8,Center for Community Change,http://n2t.net/ark:/99166/w64v22kw,13428592,
9,Central Michigan University. Office of the Pre...,http://n2t.net/ark:/99166/w6bg9nm3,46675106,From Social Networks and Archival Context:Abel...


In [53]:
# need to do some work on this one